In [6]:
import re
import cufflinks
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from nltk import word_tokenize
import plotly.graph_objs as go
import matplotlib.pyplot as plt
import nltk
from plotly.offline import iplot
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer,PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_auc_score, precision_score
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer() 

cufflinks.go_offline()
cufflinks.set_config_file(world_readable=True, theme='pearl')

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [2]:
pip install lightbgm

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement lightbgm (from versions: none)
ERROR: No matching distribution found for lightbgm


In [3]:
pip install cufflinks

In [7]:
data = pd.read_csv('Copy of Skills(1).csv')

In [8]:
df1 = data.iloc[::2]

In [9]:
df1.head()

,Title,Element Name
0,Chief Executives,Reading Comprehension
2,Chief Executives,Active Listening
4,Chief Executives,Writing
6,Chief Executives,Speaking
8,Chief Executives,Mathematics


In [10]:
data.head()

,Title,Element Name
0,Chief Executives,Reading Comprehension
1,Chief Executives,Reading Comprehension
2,Chief Executives,Active Listening
3,Chief Executives,Active Listening
4,Chief Executives,Writing


In [11]:
data=df1

In [39]:
index_names = data[ data['Title'] == "biochemists biophysicists" ].index
  
# drop these row indexes
# from dataFrame
data.drop(index_names, inplace = True)

In [40]:
data.head()

,Title,Element Name
0,chief executives,Reading Comprehension
2,chief executives,Active Listening
4,chief executives,Writing
6,chief executives,Speaking
8,chief executives,Mathematics


In [41]:
data['Title'].value_counts()

chief executives                                                        35
secretaries administrative assistants except legal medical executive    35
dispatchers except police fire ambulance                                35
meter readers utilities                                                 35
postal service clerks                                                   35
                                                                        ..
secondary school teachers except special career technical education     35
career technical education teachers secondary school                    35
special education teachers preschool                                    35
special education teachers middle school                                35
tank car truck ship loaders                                             35
Name: Title, Length: 872, dtype: int64

In [105]:
def preprocess(sentence):
    sentence = str(sentence)
    sentence = sentence.lower()
    sent1 = sentence
    cleanr = re.compile('<.*?>')
    final=re.sub("[^a-zA-Z :,]", "", sent1)
    print(final)
    cleantext = re.sub(cleanr, '', sentence)
    rem_num = re.sub('[0-9]+', '', cleantext)
    tokens=final.split(" ")
    #tok1=[]
    #for i in tokens:
     #   if(i=="and"):
      #      tok1.append(",")
       # else:
        #    tok1.append(i)
    filtered_word = [w for w in tokens if not w in stopwords.words('english')]
    stop=["interested","like","love"]
    filtered_words = [w for w in filtered_word if not w in stop]
    stem_words = [stemmer.stem(w) for w in filtered_words ]
    lemma_words = [lemmatizer.lemmatize(w) for w in stem_words]
    return " ".join(filtered_words)
    

In [43]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\charanya\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\charanya\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [44]:
data['Title'] = data['Title'].map(lambda s:preprocess(s))

In [45]:
import nltk
nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\charanya\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [46]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 30520 entries, 0 to 61108
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Title         30520 non-null  object
 1   Element Name  30520 non-null  object
dtypes: object(2)
memory usage: 715.3+ KB


In [47]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\charanya\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [48]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\charanya\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [127]:
labels = {"chief executives":0,"chief sustainability officers":1,"general operations managers":2}
labels

labels1=data.Title.unique()
labels1 

lab={}
j=0
for i in labels1:
  lab[i]=j
  j=j+1
lab  

{'chief executives': 0,
 'chief sustainability officers': 1,
 'general operations managers': 2,
 'advertising promotions managers': 3,
 'marketing managers': 4,
 'sales managers': 5,
 'administrative services managers': 6,
 'computer information systems managers': 7,
 'financial managers': 8,
 'treasurers controllers': 9,
 'investment fund managers': 10,
 'industrial production managers': 11,
 'quality control systems managers': 12,
 'geothermal production managers': 13,
 'biofuels production managers': 14,
 'biomass power plant managers': 15,
 'hydroelectric production managers': 16,
 'purchasing managers': 17,
 'transportation storage distribution managers': 18,
 'supply chain managers': 19,
 'compensation benefits managers': 20,
 'human resources managers': 21,
 'training development managers': 22,
 'farmers ranchers agricultural managers': 23,
 'construction managers': 24,
 'education childcare administrators preschool daycare': 25,
 'education administrators kindergarten secondary

In [128]:
labels = lab

In [129]:
corpus, target_labels, target_names = (data['Element Name'], [labels[label] for label in data['Title']],data['Title'])
df_new = pd.DataFrame({"text_clean":corpus,"sentiment Label": target_labels,"sentiment names": target_names})
df_new

,text_clean,sentiment Label,sentiment names
0,Reading Comprehension,0,chief executives
2,Active Listening,0,chief executives
4,Writing,0,chief executives
6,Speaking,0,chief executives
8,Mathematics,0,chief executives
...,...,...,...
61100,Systems Evaluation,871,tank car truck ship loaders
61102,Time Management,871,tank car truck ship loaders
61104,Management of Financial Resources,871,tank car truck ship loaders
61106,Management of Material Resources,871,tank car truck ship loaders


In [52]:
res=set(df_new['sentiment names'])
res

{'accountants auditors',
 'actors',
 'actuaries',
 'acupuncturists',
 'acute care nurses',
 'adapted physical education specialists',
 'adhesive bonding machine operators tenders',
 'administrative law judges adjudicators hearing officers',
 'administrative services managers',
 'adult basic education adult secondary education english second language instructors',
 'advanced practice psychiatric nurses',
 'advertising promotions managers',
 'advertising sales agents',
 'aerospace engineering operations technologists technicians',
 'aerospace engineers',
 'agents business managers artists performers athletes',
 'agricultural engineers',
 'agricultural equipment operators',
 'agricultural inspectors',
 'agricultural sciences teachers postsecondary',
 'agricultural technicians',
 'air traffic controllers',
 'aircraft cargo handling supervisors',
 'aircraft mechanics service technicians',
 'aircraft structure surfaces rigging systems assemblers',
 'airfield operations specialists',
 'airlin

In [53]:
X_train, X_test, y_train, y_test = train_test_split(np.array(df_new["text_clean"]),np.array(df_new["sentiment Label"]), test_size=0.25, random_state=0)
display(X_train.shape)
display(X_test.shape)


(22890,)

(7630,)

In [54]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\charanya\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [55]:
tfidf = TfidfVectorizer(use_idf=True, tokenizer=word_tokenize,min_df=0.00002,max_df=0.70)
X_train_tf = tfidf.fit_transform(X_train.astype('U'))
X_test_tf = tfidf.transform(X_test.astype('U'))

print(f"TF_IDF Model: Train features shape:{X_train_tf.shape} and Test features shape:{X_test_tf.shape}")

TF_IDF Model: Train features shape:(22890, 51) and Test features shape:(7630, 51)


In [24]:
#X_test_tf = vectorizer.transform(X_test)
#X_train_tf = tfidf.fit_transform(X_train.astype('U'))
feature_names_test_from_train = vectorizer.get_feature_names()
df_test_from_train = pd.DataFrame(X_test_from_tf.toarray(),columns = feature_names_test_from_train)


NameError: name 'vectorizer' is not defined

In [30]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\charanya\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [56]:
xgb = XGBClassifier(eval_metric="mlogloss",random_state=42)

In [32]:

xgb = XGBClassifier(eval_metric="mlogloss",random_state=42)
mlp = MLPClassifier(random_state=42)

clfs = {
    "XGBoost": xgb,
   
}
x1=["Troubleshootings","Science"]
x2=np.array(x1)
x_test = tfidf.transform(x2.astype('U'))

#def fit_model(clf,x_train,y_train,x_test, y_test,X11_train_tf):
model1=xgb.fit(X_train_tf,y_train)
y_pred = xgb.predict(x_test)
#accuracy = accuracy_score(y_pred, y_test)
y_pred1=model1.predict(X11_train_tf)
print(y_pred1)
#print(accuracy)
#return accuracy

accuracys = []

#for name,clf in clfs.items():
    #curr_acc = fit_model(clf,X_train_tf,y_train,X_test_tf,y_test,X11_train_tf)
    #accuracys.append(curr_acc)
#y_pred



NameError: name 'clf' is not defined

In [57]:
model1=xgb.fit(X_train_tf,y_train)

In [38]:
l1=np.array(l)
X11_train_tf = tfidf.transform(l1.astype('U'))

NameError: name 'l' is not defined

In [111]:
from keras.models import load_model
model1.save("xgbm.h5")
#y_pred=xgb.predict()

AttributeError: 'XGBClassifier' object has no attribute 'save'

In [112]:
import pickle
file_name = "xgb_reg.pkl"

# save
pickle.dump(model1, open(file_name, "wb"))

In [59]:
y_pred1=model1.predict(x_test)
print(y_pred)

[180 180]


In [29]:
models_df = pd.DataFrame({"Models":clfs.keys(),"Accuracy Scores":accuracys}).sort_values('Accuracy Scores',ascending=False)
models_df

NameError: name 'clfs' is not defined

In [102]:

x1=["Complex Problem Solving finding solutions"]
x2=np.array(x1)
X11_train_tf = tfidf.transform(x2.astype('U'))
#X_test_tf = tfidf.transform(x1.astype('U'))
#clf.fit(X_train_tf,y_train)
y_pred = model1.predict(X11_train_tf)
#accuracy = accuracy_score(y_pred, y_test)
y_pred



array([501], dtype=int64)

In [ ]:
#MODEL 2

In [149]:
df=pd.read_csv('ratings - Sheet1.csv')
df



,Unnamed: 0,User_1,User_2,User_3,User_4,User_5,User_6,User_7,User_8,User_9,User_10
0,Chief Executives,5,2,0,1,4,3,5,1,4,5
1,Chief Sustainability Officers,4,3,5,1,2,0,4,5,3,2
2,General and Operations Managers,1,4,2,3,0,5,1,2,3,0
3,Advertising and Promotions Managers,2,3,4,2,0,1,3,3,4,1
4,Marketing Managers,3,4,5,1,1,3,4,4,5,2
...,...,...,...,...,...,...,...,...,...,...,...
90,"Atmospheric, Earth, Marine, and Space Sciences...",3,2,5,5,2,2,2,3,0,1
91,"Anthropology and Archeology Teachers, Postseco...",4,5,1,2,0,0,3,3,1,1
92,"Criminal Justice and Law Enforcement Teachers,...",5,4,0,0,1,2,3,5,5,3
93,Mental Health Counselors,4,5,1,0,1,2,3,5,4,2


In [192]:
df.dtypes

Unnamed: 0    object
User_1         int64
User_2         int64
User_3         int64
User_4         int64
User_5         int64
User_6         int64
User_7         int64
User_8         int64
User_9         int64
User_10        int64
dtype: object

AttributeError: Can only use .str accessor with string values!

In [201]:
la1= ['chief executives', 'chief sustainability officers', 'general and operations managers', 'advertising and promotions managers', 'marketing managers', 'sales managers', 'administrative services managers', 'computer and information systems managers', 'financial managers', 'investment fund managers', 'quality control systems managers', 'geothermal production managers', 'biofuels production managers', 'biofuels/biodiesel technology and product development managers', 'biomass power plant managers', 'hydroelectric production managers, purchasing managers', 'transportation, storage, and distribution managers', 'supply chain managers', 'compensation and benefits managers', 'human resources managers', 'training and development managers', 'farmers, ranchers, and other agricultural managers', 'construction managers', 'wind energy operations managers', 'wind energy development managers', 'natural sciences managers', 'medical and health services managers', 'food service managers', 'lodging managers', 'social and community service managers', 'regulatory affairs managers', 'compliance managers', 'computer systems analysts', 'information security analysts', 'computer and information research scientists', 'computer network support specialists', 'computer user support specialists', 'health informatics specialists', 'computer network architects', 'telecommunications engineering specialists', 'database administrators', 'data warehousing specialists', 'network and computer systems administrators', 'computer programmers', 'software quality assurance analysts and testers', 'web developers', 'geographic information systems technologists and technicians', 'database architects', 'video game designers', 'web administrators', 'computer systems engineers/architects', 'business intelligence analysts', 'bioinformatics technicians', 'nanotechnology engineering technologists and technicians', 'intelligence analysts', 'computer hardware engineers', 'aerospace engineers', 'agricultural engineers', 'chemical engineers', 'bioengineers and biomedical engineers', 'civil engineers', 'electrical engineers', 'robotics engineers', 'education and childcare administrators, preschool and daycare', 'education administrators, kindergarten through secondary', 'education administrators, postsecondary', 'educational, guidance, and career counselors and advisors', 'health education specialists', 'directors, religious activities and education', 'education teachers, postsecondary', 'health specialties teachers, postsecondary', 'library science teachers, postsecondary', 'law teachers, postsecondary', 'social work teachers, postsecondary', 'communications teachers, postsecondary', 'geography teachers, postsecondary', 'area, ethnic, and cultural studies teachers, postsecondary', 'special education teachers, secondary school', 'adult basic education, adult secondary education, and english as a second language instructors', 'self-enrichment teachers', 'tutors', 'instructional coordinators', 'teaching assistants, postsecondary', 'art, drama, and music teachers, postsecondary', 'family and consumer sciences teachers, postsecondary', 'recreation and fitness studies teachers, postsecondary', 'adapted physical education specialists', 'librarians and media collections specialists', 'farm and home management educators', 'mathematical science teachers, postsecondary', 'atmospheric, earth, marine, and space sciences teachers, postsecondary', 'anthropology and archeology teachers, postsecondary', 'criminal justice and law enforcement teachers, postsecondary','mental health counselors','chefs and head cooks']

In [202]:
len(la1)

95

In [203]:

for i, row in df.iterrows():
    df.at[i,'Unnamed: 0'] = la1[i]

In [204]:
labelsa1=df["Unnamed: 0"].unique()
print(labelsa1) 
laba={}
j=0
for i in labelsa1:
  laba[i]=j
  j=j+1
laba 
labelas = laba
for i, row in df.iterrows():
    df.at[i,'Unnamed: 0'] = i

['chief executives' 'chief sustainability officers'
 'general and operations managers' 'advertising and promotions managers'
 'marketing managers' 'sales managers' 'administrative services managers'
 'computer and information systems managers' 'financial managers'
 'investment fund managers' 'quality control systems managers'
 'geothermal production managers' 'biofuels production managers'
 'biofuels/biodiesel technology and product development managers'
 'biomass power plant managers'
 'hydroelectric production managers, purchasing managers'
 'transportation, storage, and distribution managers'
 'supply chain managers' 'compensation and benefits managers'
 'human resources managers' 'training and development managers'
 'farmers, ranchers, and other agricultural managers'
 'construction managers' 'wind energy operations managers'
 'wind energy development managers' 'natural sciences managers'
 'medical and health services managers' 'food service managers'
 'lodging managers' 'social an

In [151]:
from sklearn.neighbors import NearestNeighbors
knn = NearestNeighbors(metric='cosine', algorithm='brute')
knn.fit(df.values)
distances, indices = knn.kneighbors(df.values, n_neighbors=3)

In [152]:
indices

array([[ 0,  6,  1],
       [ 1,  4,  3],
       [ 2,  7,  5],
       [ 3,  4,  5],
       [ 4,  3,  5],
       [ 5,  4,  3],
       [ 6,  4,  5],
       [ 7, 11, 10],
       [ 8, 17, 16],
       [ 9, 12, 27],
       [10, 14, 11],
       [11, 14, 12],
       [12, 13, 11],
       [13, 12, 11],
       [14, 19, 11],
       [15, 39, 33],
       [16, 17, 34],
       [17, 18, 16],
       [18, 34, 17],
       [19, 20, 24],
       [20, 19, 24],
       [21, 25, 32],
       [22, 32, 39],
       [23, 30, 33],
       [24, 23, 37],
       [25, 36, 27],
       [26, 38, 44],
       [27, 25, 33],
       [28, 54, 48],
       [29, 65, 45],
       [30, 31, 32],
       [31, 30, 46],
       [32, 33, 35],
       [33, 32, 41],
       [34, 39, 38],
       [35, 36, 41],
       [36, 35, 39],
       [37, 38, 43],
       [38, 44, 62],
       [39, 45, 41],
       [40, 46, 54],
       [41, 51, 46],
       [42, 68, 54],
       [43, 62, 61],
       [44, 62, 38],
       [45, 41, 46],
       [46, 51, 47],
       [47, 5

In [153]:
distances

array([[0.00000000e+00, 2.71807157e-01, 2.92155936e-01],
       [0.00000000e+00, 9.90782059e-02, 1.03945452e-01],
       [0.00000000e+00, 1.61115442e-01, 1.70041497e-01],
       [1.11022302e-16, 2.65031979e-02, 1.00346314e-01],
       [1.11022302e-16, 2.65031979e-02, 3.99936528e-02],
       [1.11022302e-16, 3.99936528e-02, 1.00346314e-01],
       [3.33066907e-16, 1.02834887e-01, 1.07465055e-01],
       [0.00000000e+00, 8.78233781e-02, 9.28249854e-02],
       [1.11022302e-16, 6.49805574e-02, 9.40343743e-02],
       [1.11022302e-16, 6.75053275e-02, 7.08529952e-02],
       [2.22044605e-16, 4.94210804e-02, 5.94491869e-02],
       [1.11022302e-16, 4.17427429e-02, 4.52310876e-02],
       [0.00000000e+00, 1.71175239e-02, 4.52310876e-02],
       [0.00000000e+00, 1.71175239e-02, 5.34522444e-02],
       [0.00000000e+00, 3.19806298e-02, 4.17427429e-02],
       [2.22044605e-16, 3.94531856e-02, 4.20205575e-02],
       [2.22044605e-16, 2.14831344e-02, 3.73710934e-02],
       [0.00000000e+00, 1.71940

In [214]:
def similarJobs(input1):
  for i in range(0,len(labelsa1)):
    if(labelsa1[i]==input1):
        index1=i
  print(index1)
  index_for_job = df.index.tolist().index(index1)

  sim_jobs = indices[index_for_job].tolist()

  job_distances = distances[index_for_job].tolist()

  id_job = sim_jobs.index(index_for_job)

  sim_jobs.remove(index_for_job)

  job_distances.pop(id_job)

  print(sim_jobs)
  
  simjob=[]
  for i in sim_jobs:
    print(labelsa1[i])
    simjob.append(labelsa1[i])
  return simjob  

In [189]:
similarJobs("Computer Systems Analysts")

32
[33, 35]
Information Security Analysts
Computer Network Support Specialists


In [163]:
similarJobs("Information Security Analysts")

33
[32, 41]
Computer Systems Analysts
Data Warehousing Specialists


In [164]:
similarJobs("Data Warehousing Specialists")

41
[51, 46]
Business Intelligence Analysts
Geographic Information Systems Technologists and Technicians


In [ ]:
similarJobs("Computer Systems Analysts")

In [157]:
similarJobs("Computer Hardware Engineers")

55
[61, 87]
Electrical Engineers
Librarians and Media Collections Specialists


In [158]:
similarJobs("Electrical Engineers")

61
[55, 52]
Computer Hardware Engineers
Bioinformatics Technicians


In [222]:
similarJobs("Nanotechnology Engineering Technologists and Technicians")

UnboundLocalError: local variable 'index1' referenced before assignment

In [236]:
ramya = { 
    "mathematical science teachers, postsecondary":"A Mathematical Science Teacher evaluates and students' class work, assignments, learning strategies and overall progress. He /she Compiles, administers, and grades tasks examinations, or delegates these tasks to others. One should also prepare and deliver lectures to undergraduate and/or graduate students on topics such as linear algebra, differential equations, and discrete mathematics; Prepare course materials such as syllabi, homework assignments, and handouts; Maintain student attendance records, grades, and other required records; Plan, evaluate, and revise curricula, course content, and course materials and methods of instruction; Initiate, facilitate, and moderate classroom discussions; Suggest career paths and milestones for long run aspirations; Keep abreast of developments in their field by reading current literature, talking with colleagues, and participating in professional conferences; Collaborate with colleagues to address teaching and research issues; Serve on academic or administrative committees that deal with institutional policies, departmental matters, and academic issues; Supervise undergraduate and/or graduate teaching, internship, and research work. Expected Salary : 1.5 to 8LPA",
    "criminal justice and law enforcement teachers, postsecondary":"A Criminal Law Enforcement Teacher should Prepare and deliver lectures to undergraduate or graduate students on topics such as criminal law, defensive policing, and investigation techniques; Participate in student recruitment, registration, and placement activities; Conduct research in a particular field of knowledge and publish findings in professional journals, books, or electronic media; Write grant proposals to procure external research funding; Identifying Objects, Actions, and Events - Identifying information by categorizing, estimating, recognizing differences or similarities, and detecting changes in circumstances or events; Establishing and Maintaining Interpersonal Relationships - Developing constructive and cooperative working relationships with others, and maintaining them over time; Processing Information - Compiling, coding, categorizing, calculating, tabulating, auditing, or verifying information or data; Scheduling Work and Activities - Scheduling events, programs, and activities, as well as the work of others; Performing for or Working Directly with the Public - Performing for people or dealing directly with the public. This includes serving customers in restaurants and stores, and receiving clients or guests; Evaluating Information to Determine Compliance with Standards - Using relevant information and individual judgment to determine whether events or processes comply with laws, regulations, or standards; Resolving Conflicts and Negotiating with Others - Handling complaints, settling disputes, and resolving grievances and conflicts; Guiding, Directing, and Motivating Subordinates - Providing guidance and direction to subordinates, including setting performance standards and monitoring performance. Expected Salary : 17 to 25LPA",
    "adapted physical education specialists" : "Besides their typical day, Adapted Physical Education Specialists also advise education professionals of students’ physical abilities or disabilities and the accommodations required to enhance their school performance. They may also assist in screening or placement of students in adapted physical education programs. On a weekly to monthly basis, Adapted Physical Education Specialists write or modify individualized education plans (ieps) for students with intellectual or physical disabilities. They might also communicate behavioral observations and student progress reports to students, parents, teachers, or administrators. In addition, they write reports to summarize student performance, social growth, or physical development. Although specific duties may vary, many of them prepare lesson plans in accordance with individualized education plans (ieps) and the functional abilities or needs of students. To some Adapted Physical Education Specialists, it is also their responsibility to evaluate the motor needs of individual students to determine their need for adapted physical education services. Expected Salary : 1.9 to 4LPA"}
dict11={
    "data warehousing specialists":"The exact tasks and job roles of a data warehousing specialist depend on their organization, as well as the scope of the project and the resources at their disposal. In general, data warehousing specialists are responsible for :\n Developing processes and procedures for data management across an organization or within the scope of a project. \n Creating software applications or designing computer programs that accomplish tasks necessary for data storage and management. \n Analyzing current data warehousing processes for improvement to make procedures more efficient, or realize a different end target. \n These job responsibilities, among others, contribute to the data warehousing system’s success. \n Data warehouse specialists’ salaries vary depending on their level of education, years of experience, and the size and industry of the company. They may also earn additional compensation in the form of bonuses.\n Median Annual Salary: $95,000 ($45.67/hour)\n Top 10% Annual Salary: $132,000 ($63.46/hour) \n The employment of data warehouse specialists is expected to grow much faster than average over the next decade.",
    "business intelligence analyst":"A business intelligence analyst, also known as a BI analyst, uses data and other information to help organizations make sound business decisions. \n Though exact job descriptions can vary, a business intelligence analyst’s role can be broadly broken down into three parts:\n Breaking down key business data: A business intelligence analyst might gather, clean, and analyze data like revenue, sales, market information, or customer engagement metrics of a business. BI analysts can also be asked to program tools and data models to help visualize or monitor data.\n Interpreting the data: Finding patterns or seeing areas in the data that signal a potential for improvement in business practices is a key part of a BI analyst’s job. For example, a BI analyst might analyze market trends to understand how a company might need to adapt its product.\n Sharing findings: Sharing findings can include anything from visualizing data in graphs and charts, to putting reports together and presenting in front of other teams or clients. Business intelligence analysts will also make recommendations to improve or grow the business based on their findings.",
    "geographic information systems technologists and technicians":"Design, program, or model Geographic Information Systems (GIS) applications or procedures.\n Collect, compile, or integrate Geographic Information Systems (GIS) data, such as remote sensing or cartographic data for inclusion in map manuscripts.\n Create, analyze, report, convert, or transfer data, using specialized applications program software.\n Document, design, code, or test Geographic Information Systems (GIS) models, internet mapping solutions, or other applications.\n Confer with users to analyze, configure, or troubleshoot applications." 
}
dict1={"mental health counselors":"A Mental Health Counselor's responsibilities include developing treatment plans, facilitating individual and group interventions and overseeing the overall mental health wellbeing of their patients . Ultimately, you will work directly with patients to help them overcome mental health issues and lead a healthier life.A mid-career Mental Health Counselor with 4-9 years of experience earns an average salary of ₹4.3 Lakhs per year , while an experienced Mental Health Counselor with 10-20 years of experience earns an average salary of ₹3.5 Lakhs per year.",
       "farm and home management educators":"A farm and home management educator may instruct on either agricultural issues such as agricultural processes and techniques, pest management, and food safety, or on home management issues such as budgeting, nutrition, and child development.Advise farmers and demonstrate techniques in areas such as feeding and health maintenance of livestock, growing and harvesting practices, and financial planning. He may Conduct classes or deliver lectures on subjects such as nutrition, home management, and farming techniques. Collaborate with producers to diagnose and prevent management and production problems. Annual Salary-0.5L-18L",
       "chefs and head cooks":"Chefs and head cooks oversee the daily food preparation at restaurants and other places where food is served . They direct kitchen staff and handle any food-related concerns. Chefs plan menus and order supplies. This role is very hands-on, requiring attention to detail and a positive work attitude.The Head Chef is responsible for ensuring that the standard and quality of the food production and the hygiene within the department is maintained at the highest level at all times. This job also includes determining production schedules and staff requirements necessary to ensure timely delivery of services. Annual Salary:0.6L-18.2L"}

In [237]:
dict1.update(dict11)
dict1.update(ramya)
print(dict1)

{'mental health counselors': "A Mental Health Counselor's responsibilities include developing treatment plans, facilitating individual and group interventions and overseeing the overall mental health wellbeing of their patients . Ultimately, you will work directly with patients to help them overcome mental health issues and lead a healthier life.A mid-career Mental Health Counselor with 4-9 years of experience earns an average salary of ₹4.3 Lakhs per year , while an experienced Mental Health Counselor with 10-20 years of experience earns an average salary of ₹3.5 Lakhs per year.", 'farm and home management educators': 'A farm and home management educator may instruct on either agricultural issues such as agricultural processes and techniques, pest management, and food safety, or on home management issues such as budgeting, nutrition, and child development.Advise farmers and demonstrate techniques in areas such as feeding and health maintenance of livestock, growing and harvesting prac

In [238]:
def prijobs(mes):
    ans=""
    for i in dict1:
        if(i==mes):
            ans=dict1[i]
    return ans        


In [ ]:
from flask import Flask, render_template, request, redirect, url_for
#from flask_ngrok import run_with_ngrok
app = Flask(__name__,template_folder='template')

@app.route('/')
def home():
    return render_template('f.html')
    #return "<h1>Hello!</h1>"
@app.route('/predict',methods=['POST','GET'])
def predict():
    message = request.form['interests']
    #sent = "i am interested in complex problem solving"
    sent1 = preprocess(message)
    l=sent1.split(',')
    l1=np.array(l)
    X11_train_tf = tfidf.transform(l1.astype('U'))
    y_pred=model1.predict(X11_train_tf)
    a=[]
    b=[]
    for i in y_pred:
        a.append(labels1[i])
    for j in a:
        if j not in b:
            b.append(j)
    print(y_pred)
    print(b)
    l=""
    for i in range(0,len(b)):
        l="".join(b[i])
        if(l=="chefs head cooks"):
            l="chefs and head cooks"
            b[i]="chefs and head cooks"
    rec=similarJobs(l)
    print(rec)
    return render_template('ans.html',
                               data1=b,data3=rec)
    return ' '.join(b)
       
@app.route('/answer',methods=['POST','GET'])
def answer():    
    message = request.form['jobs']
    print(message)
    req=prijobs(message)
    print(req)
    rec=similarJobs(message)
    print(rec)
    return render_template('job.html',
                               data1=req,data2=message)
app.run(debug=True, use_reloader=False)

#app.run()


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [08/Apr/2023 13:49:46] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [08/Apr/2023 13:49:46] "GET /css/ext.css HTTP/1.1" 404 -
127.0.0.1 - - [08/Apr/2023 13:50:00] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [08/Apr/2023 13:50:00] "GET /css/ext.css HTTP/1.1" 404 -


i am empathetic
[236]
['mental health counselors']
93
[92, 86]
criminal justice and law enforcement teachers, postsecondary
adapted physical education specialists
['criminal justice and law enforcement teachers, postsecondary', 'adapted physical education specialists']


127.0.0.1 - - [08/Apr/2023 13:50:05] "POST /answer HTTP/1.1" 200 -
127.0.0.1 - - [08/Apr/2023 13:50:05] "GET /css/ext.css HTTP/1.1" 404 -


mental health counselors
A Mental Health Counselor's responsibilities include developing treatment plans, facilitating individual and group interventions and overseeing the overall mental health wellbeing of their patients . Ultimately, you will work directly with patients to help them overcome mental health issues and lead a healthier life.A mid-career Mental Health Counselor with 4-9 years of experience earns an average salary of ₹4.3 Lakhs per year , while an experienced Mental Health Counselor with 10-20 years of experience earns an average salary of ₹3.5 Lakhs per year.
93
[92, 86]
criminal justice and law enforcement teachers, postsecondary
adapted physical education specialists
['criminal justice and law enforcement teachers, postsecondary', 'adapted physical education specialists']


...

i like cooking and time management
[470]
['chefs head cooks']
94
[88, 89]
farm and home management educators
mathematical science teachers, postsecondary
['farm and home management educators', 'mathematical science teachers, postsecondary']


127.0.0.1 - - [08/Apr/2023 13:50:28] "POST /answer HTTP/1.1" 200 -
127.0.0.1 - - [08/Apr/2023 13:50:28] "GET /css/ext.css HTTP/1.1" 404 -


chefs and head cooks
Chefs and head cooks oversee the daily food preparation at restaurants and other places where food is served . They direct kitchen staff and handle any food-related concerns. Chefs plan menus and order supplies. This role is very hands-on, requiring attention to detail and a positive work attitude.The Head Chef is responsible for ensuring that the standard and quality of the food production and the hygiene within the department is maintained at the highest level at all times. This job also includes determining production schedules and staff requirements necessary to ensure timely delivery of services. Annual Salary:0.6L-18.2L
94
[88, 89]
farm and home management educators
mathematical science teachers, postsecondary
['farm and home management educators', 'mathematical science teachers, postsecondary']


127.0.0.1 - - [08/Apr/2023 13:50:35] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [08/Apr/2023 13:50:35] "GET /css/ext.css HTTP/1.1" 404 -


i like cooking and time management
[470]
['chefs head cooks']
94
[88, 89]
farm and home management educators
mathematical science teachers, postsecondary
['farm and home management educators', 'mathematical science teachers, postsecondary']


In [247]:
sent = "i love cooking and i like time management"
sent1 = preprocess(sent)
sent1



i love cooking and i like time management


'cooking time management'

In [109]:
labels2=data["Element Name"].unique()
labels2
len(labels2)

35

In [131]:
print(labels1)

['chief executives' 'chief sustainability officers'
 'general operations managers' 'advertising promotions managers'
 'marketing managers' 'sales managers' 'administrative services managers'
 'computer information systems managers' 'financial managers'
 'treasurers controllers' 'investment fund managers'
 'industrial production managers' 'quality control systems managers'
 'geothermal production managers' 'biofuels production managers'
 'biomass power plant managers' 'hydroelectric production managers'
 'purchasing managers' 'transportation storage distribution managers'
 'supply chain managers' 'compensation benefits managers'
 'human resources managers' 'training development managers'
 'farmers ranchers agricultural managers' 'construction managers'
 'education childcare administrators preschool daycare'
 'education administrators kindergarten secondary'
 'education administrators postsecondary'
 'architectural engineering managers'
 'biofuels biodiesel technology product development

In [180]:
prijobs("mental health counselors")

mental health counselors
Farm and Home Management Educators
Chefs and Head Cooks


"A Mental Health Counselor's responsibilities include developing treatment plans, facilitating individual and group interventions and overseeing the overall mental health wellbeing of their patients . Ultimately, you will work directly with patients to help them overcome mental health issues and lead a healthier life.A mid-career Mental Health Counselor with 4-9 years of experience earns an average salary of ₹4.3 Lakhs per year , while an experienced Mental Health Counselor with 10-20 years of experience earns an average salary of ₹3.5 Lakhs per year."

In [215]:
a=similarJobs('mental health counselors')
print(a)

93
[92, 86]
criminal justice and law enforcement teachers, postsecondary
adapted physical education specialists
['criminal justice and law enforcement teachers, postsecondary', 'adapted physical education specialists']
